<a href="https://colab.research.google.com/github/azizamirsaidova/delos/blob/main/Classify_MNIST_with_Tensorlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook file classifies the **MNIST using Tensorflow** and conducts the data augmentation.

**Import dependencies.**

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np

TensorFlow version: 2.8.0


**Import MNIST dataset and split to train and test set.**

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

**Conduct data augmentation by rotating the images by 180 degrees.**

In [3]:
x_train_180 = tf.image.rot90(x_train, k=2)

In [4]:
x_test_180 = tf.image.rot90(x_test, k=2)

**Define the batch size and shuffle the dataset.**

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train_180, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test_180, y_test)).batch(32)

**Build MNIST Tensorflow Keras model with Convulational Neural Network and RELU activation function.**

In [6]:
class MNIST(Model):
  def __init__(self):
    super(MNIST, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# An instance of the model
model = MNIST()

**Sparse Categorical cross entropy loss and Adam optimizer is used.**

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()
#optimizer = tf.keras.optimizers.SGD(learning_rate=1.0)

**Evaluation Metrics used to test the performance:**
*   Loss
*   Accuracy



In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

**tf.GradientTape() is used to teain the model.**

In [9]:
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

**Model is tested.**

In [10]:
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

**MNIST Classifier trained on epoch 6 with accuracy of 98.45.**

In [13]:
EPOCHS = 6

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.003106481395661831, Accuracy: 99.88166809082031, Test Loss: 0.08041059970855713, Test Accuracy: 98.44999694824219
Epoch 2, Loss: 0.0036859021056443453, Accuracy: 99.8566665649414, Test Loss: 0.07642525434494019, Test Accuracy: 98.55999755859375
Epoch 3, Loss: 0.0024309020955115557, Accuracy: 99.92666625976562, Test Loss: 0.09809140861034393, Test Accuracy: 98.27999877929688
Epoch 4, Loss: 0.004398781340569258, Accuracy: 99.86333465576172, Test Loss: 0.07958949357271194, Test Accuracy: 98.47000122070312
Epoch 5, Loss: 0.0030536337289959192, Accuracy: 99.89666748046875, Test Loss: 0.08075742423534393, Test Accuracy: 98.5
Epoch 6, Loss: 0.00030101474840193987, Accuracy: 99.99500274658203, Test Loss: 0.09532584995031357, Test Accuracy: 98.45999908447266
